In [1]:
spark

Waiting for a Spark session to start...

Spark application_1533622723243_21490: Some(http://gw02.itversity.com:4045)

In [2]:
spark.sparkContext.getConf.getAll.foreach(x=>if(x._2.contains("/proxy/")){println(x._2)})

Waiting for a Spark session to start...

http://rm01.itversity.com:19288/proxy/application_1533622723243_21490


In [3]:
import scala.collection.JavaConversions._

In [4]:
val os_name = System.getProperty("os.name")
val hdfs_home = "/user/" + System.getProperty("user.home").split("/")(2)

os_name = Linux
hdfs_home = /user/kranthidr


/user/kranthidr

In [5]:
val path = hdfs_home+""

path = /user/kranthidr


/user/kranthidr

In [6]:
import com.cloudera.datascience.risk._
import com.cloudera.datascience.risk.RunRisk

In [7]:
import java.io.File
import java.time.LocalDate
import java.time.format.DateTimeFormatter

import scala.collection.mutable.ArrayBuffer
import org.apache.spark.mllib.stat.KernelDensity
import org.apache.spark.sql.{Dataset, SparkSession}
import org.apache.spark.sql.functions
import org.apache.spark.util.StatCounter
import breeze.plot._
import org.apache.commons.math3.distribution.ChiSquaredDistribution
import org.apache.commons.math3.distribution.MultivariateNormalDistribution
import org.apache.commons.math3.random.MersenneTwister
import org.apache.commons.math3.stat.correlation.Covariance
import org.apache.commons.math3.stat.regression.OLSMultipleLinearRegression

In [8]:
val runRisk = new RunRisk(spark)

runRisk = RunRisk(org.apache.spark.sql.SparkSession@4fb5bb20)


RunRisk(org.apache.spark.sql.SparkSession@4fb5bb20)

In [9]:
val (stocksReturnsA, factorsReturns) = runRisk.readStocksAndFactors()

stocksReturnsA = WrappedArray(Array(0.022950819672131244, 0.0, 0.015459723352319064, -0.006535947712418306, 0.021207177814029348, 0.0075187969924811904, 0.0057755775577557995, 0.0023923444976077465, 0.044887780548628506, 0.008012820512820484, 0.050986842105263094, 0.031249999999999903, 0.084703947368421, 0.04712460063897762, 0.03980099502487551, 0.028712059064807192, -0.01670644391408121, 0.0071599045346061934, 0.0182829888712242, 0.0, -0.035742035742035674, -0.03942380591357086, -0.05263157894736838, -0.019138755980861122, -0.02392344497607647, -0.029126213592232966, -0.03870458135860981, -0.032786885245901634, -0.08450704225352114, -0.040290088638195005, -0.06866614048934484, -0.041867954911433136, -0.02845528455284564, -0.022058823529411728, -0.0033333333333332624...


WrappedArray([D@1f0f5b4f, [D@5ed95ea1, [D@cd03e13, [D@78605bd8, [D@1d50a2b8, [D@968cbe5, [D@4455c7d, [D@7e8169b6, [D@66dc51bc, [D@29249961, [D@72c8e8c, [D@72158b06, [D@76406549, [D@7ba21571, [D@1819ac67, [D@468a49d9, [D@3395066c, [D@58f794f2, [D@dd8711f, [D@190835fc, [D@235f6d7, [D@6681b267, [D@281f6721, [D@70e0000b, [D@5d4ed38e, [D@363750ca, [D@65420f42, [D@1552abb9, [D@3e2bd83e, [D@32967e31, [D@475b4f3e, [D@47d42b0f, [D@27def1e3, [D@26b864df, [D@3b07fd89, [D@5ded35b5, [D@d6c9602, [D@681795ec, [D@6c78d6d2, [D@588c0675, [D@313947e0, [D@65d51cde, [D@4fb188f0, [D@77b690dd, [D@ccbcf8c, [D@41b50127, [D@280e7e9, [D@41f3af1c, [D@1396bf01, [D@7a43286d, [D@7ff92763, [D@23aefccf, [D@5e6210c, [D@6605619d, [D@50f18001, [D@4ed5654, [D@3037e984, [D@7df37f96, [D@215e09a1, [D@4fefa428, [D@409b0f98, [D@2672d7d2, [D@77b0c1bc, [D@45ccf586, [D@5d33d568, [D@6d4f555b, [D@1c83d46f, [D@3ec162bc, [D@7d9a7899, [D@6ae81ffc, [D@684a8e62, [D@5d2bd555, [D@73db20d9, [D@5966c6a1, [D@65637a71, [D@66619af5, [D@4c16a30

In [10]:
stocksReturnsA.getClass

class scala.collection.mutable.WrappedArray$ofRef

In [11]:
stocksReturnsA.length

1799

In [12]:
val stocksReturns =stocksReturnsA.drop(stocksReturnsA.length-50)

stocksReturns = WrappedArray(Array(0.1217765042979942, 0.0864022662889519, 0.12160228898426317, 0.13011695906432758, 0.12482065997130562, 0.10703812316715536, 0.17925925925925926, 0.19911504424778767, 0.024611398963730623, 0.03575989782886326, 0.04823989569752272, 0.03954081632653068, 0.09831824062095727, 0.08163265306122457, 0.09933774834437098, 0.042713567839196075, 0.0, 0.03666245259165602, 0.01972872996300865, 0.043532338308457895, 0.0012269938650306487, 0.04711425206124857, 0.11792452830188678, 0.11807228915662633, 0.11686746987951792, 0.09963099630996293, 0.09878048780487812, 0.09794437726723101, 0.06436233611442183, 0.10539215686274503, -0.0056242969628797195, -0.06329113924050629, -0.03340517241379297, -0.03451995685005397, 0.04026845637583906, 0.032186459489...


WrappedArray([D@1fdbd08, [D@5bd39361, [D@388fb33a, [D@7d9da4d4, [D@11645392, [D@566b8244, [D@1338fdd0, [D@3934067, [D@30587b44, [D@3a2ba6a9, [D@6e290a41, [D@7720ff9b, [D@4512860, [D@6328c8ad, [D@45f5ad8b, [D@35595e89, [D@3846626a, [D@1d8ea86f, [D@75ffd7a0, [D@aad6280, [D@1f3148d, [D@69d6190c, [D@621f8141, [D@34746e0, [D@1a096d79, [D@43ce094d, [D@5dc58dfc, [D@648ca0e2, [D@35bf7baf, [D@6888834, [D@77d34f74, [D@76d96adc, [D@38f505ab, [D@7a7ee3a9, [D@17bbab7d, [D@2a89acdc, [D@3ac13c7b, [D@372ff321, [D@7d0546fa, [D@1a0deaca, [D@291c3f14, [D@616267ed, [D@3607cfad, [D@43a9281d, [D@b054be3, [D@792a0861, [D@7158ca4e, [D@7b3bfdb8, [D@5ac0fb6f, [D@73008956)

In [13]:
stocksReturns.length

50

In [14]:
    //runRisk.plotDistribution(factorsReturns(2))

Name: Syntax Error.
Message: 
StackTrace: 

In [15]:
   // runRisk.plotDistribution(factorsReturns(3))

Name: Syntax Error.
Message: 
StackTrace: 

In [16]:
    val numTrials = 10000
    val parallelism = 10
    val baseSeed = 1001L

numTrials = 10000
parallelism = 10
baseSeed = 1001


1001

In [17]:
// val numTrials = 100
//     val parallelism = 10
//     val baseSeed = 1001L   

Name: Syntax Error.
Message: 
StackTrace: 

In [18]:
val trials = runRisk.computeTrialReturns(stocksReturns, factorsReturns, baseSeed, numTrials,parallelism)

trials = [value: double]


[value: double]

In [19]:
trials.cache()

[value: double]

In [20]:
trials.show()

+--------------------+
|               value|
+--------------------+
|0.001080037349040371|
|  0.0346173452407481|
|   0.012759993794705|
|0.030657992545194333|
|0.011450293420272407|
|0.001465034302548169|
|-0.01524684641389705|
| 0.05331208096165766|
|0.033709838627152294|
|-0.00297358349177...|
| 0.04732186154613125|
|-0.02049310444646962|
|-0.00197982427863...|
|-0.07589173492974757|
|0.014186193961475923|
|-0.00304341182138...|
|0.029393460994808555|
|0.017577397735689056|
|-0.03776400860932112|
|0.030530308857399005|
+--------------------+
only showing top 20 rows



In [21]:
    val valueAtRisk = runRisk.fivePercentVaR(trials)

valueAtRisk = -0.034644817100609716


-0.034644817100609716

In [22]:
    val conditionalValueAtRisk = runRisk.fivePercentCVaR(trials)

conditionalValueAtRisk = -0.04641286434568296


-0.04641286434568296

In [23]:
    println("VaR 5%: " + valueAtRisk)

VaR 5%: -0.034644817100609716


In [24]:
    println("CVaR 5%: " + conditionalValueAtRisk)

CVaR 5%: -0.04641286434568296


In [25]:
    val varConfidenceInterval = runRisk.bootstrappedConfidenceInterval(trials,runRisk.fivePercentVaR, 100, .05)

varConfidenceInterval = (-0.03586685496507723,-0.03312941707749002)


(-0.03586685496507723,-0.03312941707749002)

In [26]:
    val cvarConfidenceInterval = runRisk.bootstrappedConfidenceInterval(trials,runRisk.fivePercentCVaR, 100, .05)

cvarConfidenceInterval = (-0.0478720139315135,-0.044929612214073884)


(-0.0478720139315135,-0.044929612214073884)

In [27]:
    println("VaR confidence interval: " + varConfidenceInterval)
    println("CVaR confidence interval: " + cvarConfidenceInterval)

VaR confidence interval: (-0.03586685496507723,-0.03312941707749002)
CVaR confidence interval: (-0.0478720139315135,-0.044929612214073884)


In [28]:
    println("Kupiec test p-value: " + runRisk.kupiecTestPValue(stocksReturns, valueAtRisk, 0.05))

Kupiec test p-value: 0.0


In [29]:
//    plotDistribution(trials)

Name: Syntax Error.
Message: 
StackTrace: 